# Práctica 5: Implementación de Simplex

## Librerias

In [2]:
import numpy as np

## Simplex

Función donde se implementa el algoritmo Simplex para optimización de problemas de programación lineal en la forma estandar.

* El código fue adapatado de: https://github.com/mmolteratx/Simplex



In [ ]:
# ------------------------------------------------------------------
#                      Funciones Auxiliares
# ------------------------------------------------------------------

def printTableau(tableau, f):
  
    f = f[np.nonzero(f)]
    print("ind \t", end = "")
    for j in range(0, len(f)):
        print("x_" + str(j), end = "\t")
    for j in range(0, (len(tableau[0]) - len(f) - 2)):
        print("s_" + str(j), end = "\t")
    
    print()
    for j in range(0, len(tableau)):
        for i in range(0, len(tableau[0])):
            if(not np.isnan(tableau[j, i])):
                if(i == 0):
                    print(int(tableau[j, i]), end = "\t")
                else:
                    print(round(tableau[j, i], 2), end = "\t")
            else:
                print(end = "\t")
        print()

def getTableau(c, A, b):
  # Construct starting tableau
  c[0:len(c)] = -1 * c[0:len(c)]

  t2 = np.array([None, 0])
  numVar = len(c)
  numSlack = len(A)
    
  t2 = np.hstack(([None], c, [0]))
  
  basis = np.array([0] * numSlack)
  
  for i in range(0, len(basis)):
      basis[i] = numVar + i
      
  t1 = np.hstack((np.transpose([basis]), A, np.transpose([b])))
  
  tableau = np.vstack((t1, t2))
  
  tableau = np.array(tableau, dtype ='float')
  
  return tableau

# ------------------------------------------------------------------
#                      Funcion Simplex
# ------------------------------------------------------------------

def simplex(f, A, b, print_iter=True):
  for i in range(len(f)):
    f[i] = -1 * f[i]
    
  # Build Tableu
  tableau = getTableau(f, A, b)
      
  if(print_iter):
      print("Starting Tableau:")
      printTableau(tableau, f)
  
  # Assume initial basis is not optimal
  optimal = False

  # Keep track of iterations for display
  iter = 1

  while not optimal:
      
    if print_iter:
      print("--------------------------------------------------------------")
      print("ITERATION ", iter)
      printTableau(tableau, f)
        
    # Look for direction of decreased cost
    for cost in tableau[-1, 1:-1]:
      if cost < 0:
        optimal = False
        break
      optimal = True

    # If all directions result in decreased cost SBF is optimal
    if optimal: 
      break
      
    # nth variable enters basis, account for tableau indexing
    n = tableau[-1, 1:-1].tolist().index(np.amin(tableau[-1, 1:-1])) + 1 # PUEDE QUE NO SEA 2

    # Minimum ratio test, rth variable leaves basis 
    minimum = 99999
    r = -1
    
    for i in range(0, len(tableau)-1): 
      if (tableau[i, n] > 0):
        val = tableau[i, -1]/tableau[i, n]
        if val<minimum: 
          minimum = val 
          r = i
                    
    pivot = tableau[r, n] 
    
    print("Pivot Column:", n)
    print("Pivot Row:", r)
    print("Pivot Element: ", pivot)

    # Perform row operations 
    # Divide the pivot row with the pivot element 
    tableau[r, 1:] = tableau[r, 1:] / pivot 
    
    # Pivot other rows
    for i in range(0, len(tableau)): 
      if i != r:
        mult = tableau[i, n] / tableau[r, n]
        tableau[i, 1:] = tableau[i, 1:] - mult * tableau[r, 1:] 

    # New basic variable 
    tableau[r, 0] = n - 1
    iter += 1
      
  if print_iter:
    print("--------------------------------------------------------------")
    print("Final Tableau reached in", iter, "iterations")
    printTableau(tableau, f)
  else:
      print("Solved")
      
  x = np.array([0] * len(f), dtype = float)
  # Save coefficients
  for key in range(0, (len(tableau))):
      if (tableau[key, 0] < len(f)):
          x[int(tableau[key, 0])] = tableau[key, -1]
  
  optimalValue = -1 * tableau[-1,-1]

  print("--------------------------------------------------------------")
  print("SOLUTION: ")
  print("Fobj Optimal Value: " + str(optimalValue))
  print("Solution: " + str(x))

**Explicación del algoritmo**

<br>

El algoritmo Simplex es un método usado en la optimización para resolver problemas de programación lineal. En primera instancia, se construye una tabla mediante la función *getTableau(c, A, b)*. Nótese que en esta función en las columnas se tiene en cuenta la inclusión de las variables de holgura, por lo que se extrae el respectivo coeficiente para garantizar la restricción dada por el problema. Mientras que en cada fila se ve las ecuaciones de análisis más la función de costos a minimizar o maximizar. 


<br>

Posteriormente, luego de haber definido los coeficientes en cada casilla, se definen dos condiciones iniciales para el algoritmo: la primera es establecer un tipo de bandera que determina cuando la solución es óptima o no (variable *optimal*), esta se establece inicialmente en *False*, mientras que la segunda es la iteración en la que el algoritmo se encuentra (variable *iter*), esta se establece en 1. Este algoritmo se basa en la iteración por lo que solo debe ser ejecutado cuando no se pueda mejorar la solución (no se pueda minimizar más), es por esto que se usa un *while not optimal*, es decir, solo se ejecuta el contenido posterior mientras la variable *optimal* sea igual a *False*. Uno de los puntos relevantes de este algoritmo se ve en que se revisa el valor de los coeficientes de la función de costo para saber la dirección de sus valores, como en esta primera etapa se tienen valores negativos, se sabe que el algoritmo está en su etapa inicial y que su solución no es óptima. Como resultado, se revisa la columna pivote (aquella que posee la menor magnitud de la función Z) y se identifica la fila pivote (aquella fila que tenga el menor valor luego de dividir el coeficiente de la restricción con respecto al valor de la variable de la columna pivote). Como resultado, el elemento pivote es la intersección entre la columna y fila pivote. Se reemplaza los valores de la fila pivote por los valores de la fila entrante, que es la fila pivote dividida por el elemento pivote, mientras que el resto de filas sus valores se basan en: $Fila_{Vieja}-Coeficiente_{PivoteFila}*Fila_{Entrante}$. Cuando se finaliza, su suma en la variable *iter* un 1 para indicar que ya se culminó la iteración previa. 

<br>

Cabe resaltar que este procedimiento se repite hasta que en la función Z no exista un valor negativo, esto se verifica con un recorrido de la función Z en Tableau. Si existe un valor negativo se repite el algoritmo anterior, de lo contrario, se ejecuta un break y se establece que la variable *optimal* es True. Como todo se encuentra dentro de un while que itera mientras que la variable *optimal* sea False, el algoritmo deja de ejecutarse. Nótese que el valor óptimo se basa en evaluar en la última función de costo Z (con los nuevos coeficientes), el valor de las variables de holgura correspondiente a cada ecuación. Los coeficientes de restricción vienen a ser los valores resultantes de la solución a cada variable. Para finalizar en el algoritmo se realiza diferentes prints: para cada iteración se muestra la tabla resultante junto al elemento, fila y columna pivote. Además, se muestra el valor óptimo y la solución de cada variables.



##  Problema a)


Problema:
$$ \max f(x) = (10.000-200*10-18*70) x_1 + (8.000-200*10-18*70) x_2 $$

$$ \begin{array}{rl}
  200x_1 + 150x_2 & \leq 80.000 \\
  18x_1 + 20x_2 & \leq 9.000 \\
  x_1 & \leq 1500 \\
  x_2 & \leq 200 \\
  x_1,x_2 & \geq 0 \\
 \end{array}$$

 Forma Estándar:
 $$ \min f(x) = -6740 x_1 - 5100 x_2  $$

$$ \begin{array}{rl}
  200x_1 + 150x_2  + s_1 & = 80.000 \\
  18x_1 + 20x_2 + s_2 &= 9.000 \\
  x_1 + s_3 = 1500 \\
  x_2 + s_4 = 200 \\
  x_1,x_2 & \geq 0 \\
 \end{array}$$

In [7]:
# Probar función
f1 = np.array([-6740, -5100, 0, 0, 0, 0])
A1 = np.array([[200, 150, 1, 0, 0, 0], [18, 20, 0, 1, 0, 0], [1, 0, 0, 0, 1, 0], [0, 1, 0, 0, 0, 1]])
b1 = np.array([80000, 9000, 1500, 200])

simplex(f1,A1,b1)

Starting Tableau:
ind 	x_0	x_1	s_0	s_1	s_2	s_3	
6	200.0	150.0	1.0	0.0	0.0	0.0	80000.0	
7	18.0	20.0	0.0	1.0	0.0	0.0	9000.0	
8	1.0	0.0	0.0	0.0	1.0	0.0	1500.0	
9	0.0	1.0	0.0	0.0	0.0	1.0	200.0	
	-6740.0	-5100.0	0.0	0.0	0.0	0.0	0.0	
--------------------------------------------------------------
ITERATION  1
ind 	x_0	x_1	s_0	s_1	s_2	s_3	
6	200.0	150.0	1.0	0.0	0.0	0.0	80000.0	
7	18.0	20.0	0.0	1.0	0.0	0.0	9000.0	
8	1.0	0.0	0.0	0.0	1.0	0.0	1500.0	
9	0.0	1.0	0.0	0.0	0.0	1.0	200.0	
	-6740.0	-5100.0	0.0	0.0	0.0	0.0	0.0	
Pivot Column: 1
Pivot Row: 0
Pivot Element:  200.0
--------------------------------------------------------------
ITERATION  2
ind 	x_0	x_1	s_0	s_1	s_2	s_3	
0	1.0	0.75	0.0	0.0	0.0	0.0	400.0	
7	0.0	6.5	-0.09	1.0	0.0	0.0	1800.0	
8	0.0	-0.75	-0.0	0.0	1.0	0.0	1100.0	
9	0.0	1.0	0.0	0.0	0.0	1.0	200.0	
	0.0	-45.0	33.7	0.0	0.0	0.0	2696000.0	
Pivot Column: 2
Pivot Row: 3
Pivot Element:  1.0
--------------------------------------------------------------
ITERATION  3
ind 	x_0	x_1	s_0	s_1	s_2	

##  Problema b)

Problema:
$$ \max f(x) = 1.1 x_1 + 1.0 x_2 + 0.9 x_3 + 0.92 x_4 + 1.2 x_5 $$

$$ \begin{array}{rl}
  0.45x_1 + 0.5x_2 + 0.4x_3 + 0.4x_4 + 0.42x_5 & \leq 240 \\
  0.5x_1 + 0.4x_2 + 0.4x_3 + 0.4x_4 + 0.2x_5 & \leq 190 \\
  0.1x_1 + 0.15x_2 + 0.3x_3 + 0.2x_4 + 0.1x_5 & \leq 80 \\
  x_1,x_2,x_3,x_4,x_5 & \geq 0 \\
 \end{array}$$

 Forma Estándar:
 $$ \min f(x) = -1.1 x_1 - 1.0 x_2 - 0.9 x_3 - 0.92 x_4 - 1.2 x_5  $$

$$ \begin{array}{rl}
  0.45x_1 + 0.5x_2 + 0.4x_3 + 0.4x_4 + 0.42x_5 + s_1 & = 240 \\
  0.5x_1 + 0.4x_2 + 0.4x_3 + 0.4x_4 + 0.2x_5 + s_2 & = 190 \\
  0.1x_1 + 0.15x_2 + 0.3x_3 + 0.2x_4 + 0.1x_5 + s_3 & =80 \\
  x_1,x_2,x_3,x_4,x_5 & \geq 0 \\
 \end{array}$$

In [8]:
# Probar función
f2 = np.array([-1.1, -1.0, -0.9, -0.92, -1.2, 0, 0, 0])
A2 = np.array([[0.45, 0.5, 0.4, 0.4, 0.42, 1, 0, 0], [0.5, 0.4, 0.4, 0.4, 0.2, 0, 1, 0], [0.1, 0.15, 0.3, 0.2, 0.1, 0, 0, 1]])
b2 = np.array([240, 190, 80])

simplex(f2,A2,b2)

Starting Tableau:
ind 	x_0	x_1	x_2	x_3	x_4	s_0	s_1	s_2	
8	0.45	0.5	0.4	0.4	0.42	1.0	0.0	0.0	240.0	
9	0.5	0.4	0.4	0.4	0.2	0.0	1.0	0.0	190.0	
10	0.1	0.15	0.3	0.2	0.1	0.0	0.0	1.0	80.0	
	-1.1	-1.0	-0.9	-0.92	-1.2	0.0	0.0	0.0	0.0	
--------------------------------------------------------------
ITERATION  1
ind 	x_0	x_1	x_2	x_3	x_4	s_0	s_1	s_2	
8	0.45	0.5	0.4	0.4	0.42	1.0	0.0	0.0	240.0	
9	0.5	0.4	0.4	0.4	0.2	0.0	1.0	0.0	190.0	
10	0.1	0.15	0.3	0.2	0.1	0.0	0.0	1.0	80.0	
	-1.1	-1.0	-0.9	-0.92	-1.2	0.0	0.0	0.0	0.0	
Pivot Column: 5
Pivot Row: 0
Pivot Element:  0.42
--------------------------------------------------------------
ITERATION  2
ind 	x_0	x_1	x_2	x_3	x_4	s_0	s_1	s_2	
4	1.07	1.19	0.95	0.95	1.0	2.38	0.0	0.0	571.43	
9	0.29	0.16	0.21	0.21	0.0	-0.48	1.0	0.0	75.71	
10	-0.01	0.03	0.2	0.1	0.0	-0.24	0.0	1.0	22.86	
	0.19	0.43	0.24	0.22	0.0	2.86	0.0	0.0	685.71	
--------------------------------------------------------------
Final Tableau reached in 2 iterations
ind 	x_0	x_1	x_2	x_3	x_4	s_0	s_1	s_2	
